### Simulation parameters

In [1]:
suppressPackageStartupMessages(library(meta))

Niter <- 1000
Nstud <- 10
SampleSizes <- 5:40
mu1 <- 10
mu2 <- 10
sigma1 <- 1.2
sigma2 <- 1

### Samples generation

In [2]:
n1 <- sample(SampleSizes, Nstud, rep=T)
n2 <- sample(SampleSizes, Nstud, rep=T)

ind1 <- lapply(n1, rnorm, mu1, sigma1)
ind2 <- lapply(n2, rnorm, mu2, sigma2)
sd1 <- sapply(ind1, sd)
sd2 <- sapply(ind2, sd)

### F-test for comparison two variances

In [3]:
Fr <- sd1^2 / sd2^2
df1 <- n1 - 1
df2 <- n2 - 1
f.pvalues <- 1 - abs(pf(Fr, df1, df2) - 0.5) * 2
alpha <- .05
ci <- cbind(Fr/qf(1-alpha/2, df1, df2), Fr/qf(alpha/2, df1, df2))
data.frame(study=1:Nstud, cases=n1, controls=n2, estimate=Fr, lower.95=ci[,1], upper.95=ci[,2], pval=f.pvalues)

study,cases,controls,estimate,lower.95,upper.95,pval
1,16,22,4.1501208,1.6379281,11.372455,0.003092150
2,6,32,0.3452420,0.1146862,2.147484,0.237312883
3,7,40,1.1383502,0.4134045,5.710600,0.717454575
4,38,5,10.2202825,1.2133233,32.252427,0.034996704
5,33,11,1.4698504,0.4457828,3.645144,0.531408719
6,37,14,1.2362510,0.4416655,2.829775,0.705161537
7,12,13,0.3937567,0.1185485,1.350433,0.133459604
8,21,21,1.3180122,0.5348024,3.248220,0.542647267
9,34,35,2.5166285,1.2658033,5.021387,0.008959227
10,19,32,2.6952598,1.2158358,6.567933,0.014859527


### Meta-analysis on ratios of variances

In [4]:
TE <- log(Fr) - digamma(df1/2) + digamma(df2/2) + log(df1/df2)
TEse <- sqrt(trigamma(df1/2) + trigamma(df2/2))
meta <- metagen(TE, TEse)
cat(sprintf("Variance ratio estimate (real value: %g):", sigma1^2/sigma2^2))
with(meta, data.frame(method=c("fixed", "random"), estimate=exp(c(TE.fixed, TE.random)/2),
        lower.95=exp(c(lower.fixed, lower.random)/2),
        upper.95=exp(c(upper.fixed, upper.random)/2), pval=c(pval.fixed, pval.random)))

Variance ratio estimate (real value: 1.44):

method,estimate,lower.95,upper.95,pval
fixed,1.325345,1.131698,1.552128,0.0004738349
random,1.282319,1.005897,1.634702,0.0447042950


### Monte Carlo simulation

In [5]:
metas <- replicate(Niter, {
	n1 <- sample(SampleSizes, Nstud, rep=T)
	n2 <- sample(SampleSizes, Nstud, rep=T)
	
	ind1 <- lapply(n1, rnorm, mu1, sigma1)
	ind2 <- lapply(n2, rnorm, mu2, sigma2)
	sd1 <- sapply(ind1, sd)
	sd2 <- sapply(ind2, sd)
	
	Fr <- sd1^2 / sd2^2
	df1 <- n1 - 1
	df2 <- n2 - 1
    
	TE <- log(Fr) - digamma(df1/2) + digamma(df2/2) + log(df1/df2)
	TEse <- sqrt(trigamma(df1/2) + trigamma(df2/2))
	metagen(TE, TEse)
}, simplify=F)

### Distribution of estimations

In [6]:
cat(sprintf("Real log-variance ratio: %g", 2*log(sigma1/sigma2)))
TEs.fixed <- sapply(metas, "[[", "TE.fixed")
TEs.random <- sapply(metas, "[[", "TE.random")
data.frame(mean = c(mean(TEs.fixed), mean(TEs.random)), sd=c(sd(TEs.fixed), sd(TEs.random)),
            sem = c(sd(TEs.fixed)/sqrt(length(TEs.fixed)),
            sd(TEs.random)/sqrt(length(TEs.random))), row.names=c("fixed", "random"))

Real log-variance ratio: 0.364643

,mean,sd,sem
fixed,0.3682625,0.1597319,0.005051166
random,0.3691706,0.1611201,0.005095065


### Power of estimations (rate of log-variance ratios estimated significantly different from 0)

In [7]:
c(fixed = mean(sapply(metas, "[[", "pval.fixed") <= .05), random = mean(sapply(metas, "[[", "pval.random") <= .05))

fixed random 
 0.666  0.600

### Frequency of incorrect estimations (Rate of estimated 95% confidence intervals not including real value)

In [8]:
lvr <- 2*log(sigma1/sigma2)
c(fixed = mean(sapply(metas, "[[", "lower.fixed") > lvr | sapply(metas, "[[", "upper.fixed") < lvr),
 random = mean(sapply(metas, "[[", "lower.random") > lvr | sapply(metas, "[[", "upper.random") < lvr))

fixed random 
 0.060  0.044